<a href="https://colab.research.google.com/github/Shai2u/BatYamNY/blob/master/population_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#Installing specific packages.
!pip install dash==2.0.0
!pip install jupyter_dash
!pip install geopandas
!pip install dash-leaflet==0.1.23
!pip install dash-extensions==0.0.65

In [21]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, html, dcc, Input, Output
from dash_extensions.javascript import  assign

import dash_leaflet.express as dlx
import plotly.express as px
import plotly.graph_objects as go
import dash_leaflet as dl
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from jupyter_dash import JupyterDash

In [22]:
#pandas configuration
pd.options.display.max_columns=100

# Loading Files

In [23]:

#Statistical Stats
statisticalStatsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/statistical_tract_4326.geojson')
statisticalStatsJson = json.loads(statisticalStatsGdf.to_json())
#building simulation
simulatedBldgsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
simulatedBldgsJson = json.loads(simulatedBldgsGdf.to_json())
simulatedBldgsGdf['start_date'] = pd.to_datetime(simulatedBldgsGdf['start_date'])
simulatedBldgsGdf['end_date'] = pd.to_datetime(simulatedBldgsGdf['end_date'])
#agents track
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')
agents_track_status.drop(columns='Unnamed: 0',inplace=True)

agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

year_ranges = agents_stat_summary_by_year['year']

attribution = '© OpenStreetMap contributors, © CARTO'
cartoUrl = 'http://basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'

# style =   color: 'red',
#                 weight: 10,
#                 opacity: .7,
#                 dashArray: '20,15',
#                 lineJoin: 'round'


## Pre-processing

In [24]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [25]:
# Generate count status graph
def get_status_graph(bldg_status):
  fig = px.bar(bldg_status, x="status", y="count",width=700,height=200,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  font=dict(
      size=15,
  )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig


# Dot Matrix

In [30]:

    #fig.write_image(f'matrixPlot_{ticNumber}.png')

#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])

list_ = ['Staying']* agents_time_new_Comers_stay.loc[20,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[20,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[20,'emptyPalces'].astype(int)
dotMAtrixFig1 = exportFigure_sq(list_,('temp'),MatrixPlot)



In [58]:
dotMAtrixFig1

In [57]:

def exportFigure_sq(list_,date_,MatrixPlot):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=str(date_),
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    fig.update_layout(width=500,height=500,margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=6))

    return fig

# Dash

In [131]:

def prepare_dot_matrix(q_date_for_dot_matrix):
    agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

    agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
    agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
    agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
    agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
    agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
    agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

    agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
    agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
    agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']

    #create an empty dataset with agent coutn
    num_of_agents = 3481
    MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
    #Number of Columns choose:
    numOfColumns=59
    numOfRows = 59
    # set the columns to display the column number
    MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
    #prepare the y column (row number)
    y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
    temp = []
    for i in range(0,numOfRows):
        temp +=y[i]
    y = temp
    MatrixPlot['y'] = y
    agents_stat_short = agents_stat_summary_by_year[['year','New Comers','stay']].copy().fillna(0)
    agents_stat_short['emptyPalces'] =agents_stat_short.apply(lambda p:num_of_agents-(p['stay']+p['New Comers']), axis=1)
    stay_count = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'stay'].values[0])
    new_comers = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'New Comers'].values[0])
    empty_space = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'emptyPalces'].values[0])
    list_ = ['Staying']* stay_count
    list_ = list_ + ['New Comers'] * new_comers
    list_ = list_ + ['Future Units'] * empty_space
    dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix),MatrixPlot)
    return dotMAtrixFig
    #dotMAtrixFig = dotMAtrixFig1

line_style = dict(weight=2, opacity=1, color='blue', fillOpacity=0,dashArray="10 10")

classes = ['Building before', 'Construction', 'Building after']
colorscale = ['#FFEDA0', '#FEB24C', '#FC4E2A']
style = dict(weight=2, opacity=1, color='white', fillOpacity=0.7)
colorbar = dlx.categorical_colorbar(categories=classes, colorscale=colorscale, width=300, height=30, position="bottomleft")

style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value == classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")


mapObj = dl.Map([dl.TileLayer(url=cartoUrl, maxZoom=20, attribution=attribution),dl.GeoJSON(data=statisticalStatsJson, options={'style':line_style}),dl.GeoJSON(data = simulatedBldgsJson,options = {'style':style_handle}, id='simulatedBldgs',hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="status")),colorbar],center=[32.0272,34.7444], zoom=16, style={'width': '100%', 'height': '900px'})


d = pd.to_datetime('2022-01-01')
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
bldg_status_count = get_status_graph(bldg_status)


#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])


dashboard_page =  html.Div([
  html.Div([
    html.Div([
              #To Do Add static legend, that won't chnaged based on years and statistical stats
              html.Div(['Place2',mapObj
              
              ])
            ],style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            html.Table([
                html.Tr([html.Td(['Place1'],id='selectedDate',style={'direction':'rtl','width':'10%'}),
                    html.Td([
                                    dcc.Slider(id='years-slider',
             min=2015,
             max=2030,
             value=2022,
             marks={str(year):str(year) for year in np.arange(2015,2030,1)},
                                                    step=0.5
                                                    )
                    ],style={'direction':'rtl','width':'90%'})
                    
                ])
            ],style={'direction':'rtl','width':'100%'}),html.Div([dcc.Graph(id='status_count',figure=bldg_status_count)]),
            html.Div([
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div([dcc.Graph(id='dot_matrix_fig',figure=dotMAtrixFig1),html.Div(id='test',children='test')])],style={'width': '49%', 'display': 'inline-block'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                                html.Div(['Hello World'])],style={'width': '49%', 'display': 'inline-block'})

            ])




                                                    ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
  ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'})
        #dcc.Store(id='eng_heb_data')
])
app = JupyterDash(__name__,suppress_callback_exceptions=True,prevent_initial_callbacks=True)
app.layout = html.Div([

    dashboard_page
])

@app.callback(
    Output('selectedDate', 'children'),
    Output('simulatedBldgs', 'data'),
    Output('status_count','figure'),
    Output('dot_matrix_fig','figure'),
    Input('years-slider', 'value')
    # Output('test','children'),
    #Output('dot_matrix_fig','figure'),

)
def update_output_div(input_value):
    #simulatedBldgsGdf
    date_return = f'Q3 {int(input_value)}'
    q_date_for_dot_matrix = f'{int(input_value)} Q3'
    d = pd.to_datetime(f'{int(input_value)}-07-01')
    if input_value % 1 == 0:
        d = pd.to_datetime(f'{int(input_value)}-01-01')
        date_return = f'Q1 {int(input_value)}'
        q_date_for_dot_matrix = f'{int(input_value)} Q1'
    
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)
    bldgsJson = json.loads(bldgs.to_json())


    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
    bldg_status_count = get_status_graph(bldg_status)


    # Dot Matrix


    #create an empty dataset with agent coutn
    num_of_agents = 3481
    MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
    #Number of Columns choose:
    numOfColumns=59
    numOfRows = 59
    # set the columns to display the column number
    MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
    #prepare the y column (row number)
    y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
    temp = []
    for i in range(0,numOfRows):
        temp +=y[i]
    y = temp
    MatrixPlot['y'] = y
    agents_stat_short = agents_stat_summary_by_year[['year','New Comers','stay']].copy().fillna(0)
    agents_stat_short['emptyPalces'] =agents_stat_short.apply(lambda p:num_of_agents-(p['stay']+p['New Comers']), axis=1)
    stay_count = int(agents_stat_short.query(f'year=="{q_date_for_dot_matrix}"')['stay'].values[0])
    new_comers = int(agents_stat_short.query(f'year=="{q_date_for_dot_matrix}"')['New Comers'].values[0])
    empty_space = int(agents_stat_short.query(f'year=="{q_date_for_dot_matrix}"')['emptyPalces'].values[0])

    list1_ = ['Staying']* stay_count
    list1_ = list1_ + ['New Comers'] * new_comers
    list1_ = list1_ + ['Future Units'] * empty_space


    #Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list1_
    dotMAtrixFig1 = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=str(q_date_for_dot_matrix),
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    dotMAtrixFig1.update_layout(width=500,height=500,margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    dotMAtrixFig1.update_yaxes(showticklabels=False,visible=False)
    dotMAtrixFig1.update_xaxes(showticklabels=False,visible=False)
    dotMAtrixFig1.update_traces(marker=dict(size=6))
    #dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix),MatrixPlot)

    return date_return,bldgsJson,bldg_status_count,dotMAtrixFig1

## To dos
## Add Tama 38 Construction Current Coutner
app.run_server(mode='external',debug=False,port=8050)

Dash app running on:


<IPython.core.display.Javascript object>

In [66]:

prepare_dot_matrix('2021 Q3')

In [106]:
agents_stat_summary_by_year[agents_stat_summary_by_year['year']==str('2019 Q3')]['stay'].values[0]

764.0

In [109]:
agents_stat_summary_by_year.query('year=="2019 Q3"')['stay']

9    764.0
Name: stay, dtype: float64

In [85]:
agents_stat_summary_by_year[agents_stat_summary_by_year['year']==str('2019 Q3')]

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
9,0,2019 Q3,82.0,66.0,82.0,73.0,102.0,80.0,32.0,34.0,46.0,46.5,62.0,69.0,19415.105263,6503.75,21815.473684,8965.0,25173.131579,11886.75,90.0,271,195.0,493,0.0,384.0,75.0,380.0,210.0,0.0,285.0,764.0,1049.0,0.0,0.071497,0.200191,0.366063,0.36225,0.0


In [127]:
['Staying']* 22.0

TypeError: ignored

In [ ]:
agents_time_new_Comers_stay.head()

,year,New Comers,stay,emptyPalces
0,2015 Q1,0.0,897.0,2584.0
1,2015 Q3,0.0,875.0,2606.0
2,2016 Q1,0.0,875.0,2606.0
3,2016 Q3,0.0,836.0,2645.0
4,2017 Q1,25.0,802.0,2654.0


In [ ]:
    list_ = ['Staying']* int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']== '2015 Q1','stay'].values[0])
    list_ = list_ + ['New Comers']*int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']== '2015 Q1','New Comers'].values[0])
    list_ = list_ + ['Future Units']*int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']== '2015 Q1','emptyPalces'].values[0])
    dotMAtrixFig = exportFigure_sq(list_,('2015 Q1'))

In [ ]:
    q_date_for_dot_matrix = '2017 Q3'
    list_ = ['Staying']* int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']==q_date_for_dot_matrix,'stay'].values[0])
    list_ = list_ + ['New Comers']*int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']== q_date_for_dot_matrix,'New Comers'].values[0])
    list_ = list_ + ['Future Units']*int(agents_time_new_Comers_stay.loc[agents_time_new_Comers_stay['year']== q_date_for_dot_matrix,'emptyPalces'].values[0])
    dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix))

In [ ]:
dotMAtrixFig

# After Dash

In [ ]:
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

NameError: ignored

In [ ]:
bldgs1[['project_nu','status']]

NameError: ignored

In [ ]:
agents_track_status_current = pd.merge(agents_track_status,bldgs1[['project_nu','status']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])

NameError: ignored

In [ ]:
agents_track_status_current

NameError: ignored

In [ ]:
agents_stat_summary_by_year.head()

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6403.0,NaN,8771.0,NaN,11667.00,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0.0,NaN,897.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,66.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,65.25,NaN,6427.0,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0.0,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Apartment Size  (q1-q3) vs Staying or New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="age (q1,q2,q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Average Age"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Icome Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Income (q1-q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"})

fig.show()

# set range

In [ ]:
agents_stat_summary_by_year.head()

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6403.0,NaN,8771.0,NaN,11667.00,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0.0,NaN,897.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,66.00,NaN,6413.5,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,65.25,NaN,6427.0,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0.0,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_rent'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Renters",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_own'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Owners",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_rent'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Renters",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_own'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Owners",
    mode="lines",
    line=dict(color='firebrick', width=5)
))



fig.update_layout(title="Income By Class",template='plotly_white',yaxis = {'title' : "Absolute Numbers"})

fig.show()

# set range

Their is a lot of ownown regrading how the population will look like because this is a complex matter

In [ ]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_low_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Low Inocme",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_medium_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Medium Income",
    mode="lines",
    line=dict(color='royalblue', width=3)
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_high_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="High Income",
    mode="lines",
    line=dict(color='royalblue', width=5)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_low_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Renters",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_medium_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Medium",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_high_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="High",
    mode="lines",
    line=dict(color='firebrick', width=5)
))


fig.update_layout(title="Income by class",template='plotly_white',yaxis = {'title' : "ratio"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="New comers",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Staying",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.update_layout(title="Number of People",template='plotly_white',yaxis = {'title' : "Absolute"})

fig.show()

# set range

## Dot Matrix

In [ ]:

def exportFigure(list_,ticNumber):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=str(ticNumber),
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    fig.update_layout(width=700,height=1200,margin=dict(l=50, r=50, t=100, b=50),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=8))
    return fig
    #fig.write_image(f'matrixPlot_{ticNumber}.png')


# marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey')),
#                   selector=dict(mode='markers')
#create an empty dataset with agent coutn
MatrixPlot = pd.DataFrame({'AgentID':range(0,3400)})
#Number of Columns choose:
numOfColumns=40
numOfRows = 85
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = 3400 -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])

i=20
list_ = ['Staying']* agents_time_new_Comers_stay.loc[i,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[i,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[i,'emptyPalces'].astype(int)
dotMAtrixFig = exportFigure(list_,(i))



In [ ]:
MatrixPlot

,AgentID,x,y
0,0,1,1
1,1,2,1
2,2,3,1
3,3,4,1
4,4,5,1
...,...,...,...
3395,3395,36,85
3396,3396,37,85
3397,3397,38,85
3398,3398,39,85


In [ ]:
aggResByTic = pd.read_excel('https://raw.githubusercontent.com/Shai2u/BatYamNY/master/excel/aggregated_results_by_project_april17.xlsx')

In [ ]:
stayNew = aggResByTic[['tic','stay','new comers']].fillna(0)

In [ ]:
stayNew['emptyPalces'] = 3400-(stayNew['stay'] +stayNew['new comers'])

In [ ]:
agents_time_new_Comers_stay.head()

,year,New Comers,stay,emptyPalces
0,2015 Q1,0.0,897.0,2503.0
1,2015 Q3,0.0,875.0,2525.0
2,2016 Q1,0.0,875.0,2525.0
3,2016 Q3,0.0,836.0,2564.0
4,2017 Q1,25.0,802.0,2573.0


In [ ]:
stayNew.head()

,tic,stay,new comers,emptyPalces
0,0,897,0.0,2503.0
1,1,884,34.0,2482.0
2,2,868,60.0,2472.0
3,3,854,91.0,2455.0
4,4,841,118.0,2441.0


In [ ]:
897+2503

3400

In [ ]:
agents_time_new_Comers_stay.head()

,year,New Comers,stay,emptyPalces
0,2015 Q1,0.0,897.0,2503.0
1,2015 Q3,0.0,875.0,2525.0
2,2016 Q1,0.0,875.0,2525.0
3,2016 Q3,0.0,836.0,2564.0
4,2017 Q1,25.0,802.0,2573.0


In [ ]:
agents_time_new_Comers_stay.loc[0,'stay'].astype(int)

897

In [ ]:
MatrixPlot['Who'] = 'stay'

In [ ]:
MatrixPlot.head()

,AgentID,x,y,Who
0,0,1,1,Staying
1,1,2,1,Staying
2,2,3,1,Staying
3,3,4,1,Staying
4,4,5,1,Staying


In [ ]:
i=20
list_ = ['Staying']* agents_time_new_Comers_stay.loc[i,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[i,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[i,'emptyPalces'].astype(int)
exportFigure(list_,(i))

In [ ]:
ticlist = []
for i in agents_time_new_Comers_stay.index:
    list_ = ['Staying']* agents_time_new_Comers_stay.loc[i,'stay'].astype(int)
    list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[i,'New Comers'].astype(int)
    list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[i,'emptyPalces'].astype(int)
    ticlist.append(exportFigure(list_,(i)))

In [ ]:
ticlist = []
for i in range(1,32):
    list_ = ['Staying']* stayNew.loc[i,'stay']
    list_ = list_ + ['New Comers']*stayNew.loc[i,'new comers'].astype(int)
    list_ = list_ + ['Future Units']*stayNew.loc[i,'emptyPalces'].astype(int)
    ticlist.append(exportFigure(list_,(i)))

In [ ]:

def exportFigure(list_,ticNumber):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=str(ticNumber),
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "orange",
                         "New Comers": "blue",
                         "Future Units": "hsv(0,0%,95%)"})
    fig.update_layout(width=700,height=1200,margin=dict(l=50, r=50, t=100, b=50),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=8))
    return fig
    #fig.write_image(f'matrixPlot_{ticNumber}.png')


# marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey')),
#                   selector=dict(mode='markers')

In [ ]:
!pip install -U kaleido

     |████████████████████████████████| 79.9 MB 144 kB/s 


In [ ]:
agents_stat_summary_by_year

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,70.0,NaN,73.0,NaN,73.0,NaN,30.00,NaN,43.0,NaN,65.0,NaN,6392.00,NaN,8028.0,NaN,9827.75,NaN,1573,NaN,2259,NaN,2447.0,NaN,1385.0,NaN,0.0,NaN,3832.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2015 Q3,NaN,73.0,NaN,73.0,NaN,73.0,NaN,30.00,NaN,43.0,NaN,65.0,NaN,6392.00,NaN,8028.0,NaN,9827.00,NaN,1565,NaN,2245,NaN,2431.0,NaN,1379.0,NaN,0.0,NaN,3810.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2016 Q1,NaN,73.0,NaN,73.0,NaN,73.0,NaN,31.00,NaN,44.0,NaN,66.0,NaN,6392.00,NaN,8028.0,NaN,9827.00,NaN,1565,NaN,2245,NaN,2431.0,NaN,1379.0,NaN,0.0,NaN,3810.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2016 Q3,NaN,73.0,NaN,73.0,NaN,73.0,NaN,31.00,NaN,44.0,NaN,66.0,NaN,6398.00,NaN,8028.0,NaN,9827.00,NaN,1549,NaN,2222,NaN,2409.0,NaN,1362.0,NaN,0.0,NaN,3771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017 Q1,82.0,73.0,82.0,73.0,82.0,73.0,32.00,32.00,47.0,45.0,60.00,67.0,18838.105263,6398.00,21097.105263,8039.0,22033.947368,9830.00,11.0,1537,14.0,2200,0.0,2382.0,11.0,1355.0,14.0,0.0,25.0,3737.0,3762.0,0.0,0.002924,0.003721,0.633174,0.360181,0.0
5,5,2017 Q3,82.0,73.0,82.0,73.0,82.0,73.0,32.00,32.00,47.0,45.0,60.00,67.0,18838.105263,6398.00,21097.105263,8039.0,22033.947368,9830.00,11.0,1537,14.0,2200,0.0,2382.0,11.0,1355.0,14.0,0.0,25.0,3737.0,3762.0,0.0,0.002924,0.003721,0.633174,0.360181,0.0
6,6,2018 Q1,82.0,73.0,82.0,73.0,92.0,73.0,31.75,33.00,42.0,46.0,60.25,68.0,19619.105263,6399.25,21099.868421,8039.0,22017.447368,9827.00,31.0,1533,53.0,2197,0.0,2379.0,20.0,1351.0,64.0,0.0,84.0,3730.0,3814.0,0.0,0.005244,0.016780,0.623755,0.354221,0.0
7,7,2018 Q3,72.0,73.0,82.0,73.0,92.0,73.0,31.00,33.00,44.0,46.0,59.00,68.0,18584.605263,6399.25,20230.421053,8039.0,22263.842105,9827.00,69.0,1533,134.0,2197,0.0,2379.0,75.0,1351.0,128.0,0.0,203.0,3730.0,3933.0,0.0,0.019069,0.032545,0.604882,0.343504,0.0
8,8,2019 Q1,82.0,73.0,82.0,73.0,102.0,73.0,32.00,34.00,46.0,47.0,62.00,69.0,19415.105263,6403.00,21815.473684,8039.0,25173.131579,9827.00,90.0,1526,195.0,2173,0.0,2361.0,75.0,1338.0,210.0,0.0,285.0,3699.0,3984.0,0.0,0.018825,0.052711,0.592620,0.335843,0.0
9,9,2019 Q3,82.0,73.0,82.0,73.0,102.0,73.0,32.00,34.00,46.0,47.0,62.00,69.0,19415.105263,6403.00,21815.473684,8039.0,25173.131579,9827.00,90.0,1526,195.0,2173,0.0,2361.0,75.0,1338.0,210.0,0.0,285.0,3699.0,3984.0,0.0,0.018825,0.052711,0.592620,0.335843,0.0
